## SoRec

In [1]:
import numpy as np

In [2]:
# toy data

G=np.array([[0.0,0.0,0.0,0.0,0.0,0.0],
            [0.0,0.0,0.0,1.0,0.0,0.8],
            [0.8,0.0,0.0,0.0,0.0,0.0],
            [0.8,1.0,0.0,0.0,0.6,0.0],
            [0.0,0.0,0.4,0.0,0.0,0.8],
            [0.0,0.0,0.0,0.0,0.0,0.0]])

R=np.array([[5.0,2.0,0.0,3.0,0.0,4.0,0.0,0.0],
            [4.0,3.0,0.0,0.0,5.0,0.0,0.0,0.0],
            [4.0,0.0,2.0,0.0,0.0,0.0,2.0,4.0],
            [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
            [5.0,1.0,2.0,0.0,4.0,3.0,0.0,0.0],
            [4.0,3.0,0.0,2.0,4.0,0.0,3.0,5.0]])

In [3]:
num_user=6
num_item=8
dim=5

In [4]:
def predict(i,j):
    return U[i,:]@V[j,:].T

In [5]:
def predictc(i,k):
    return U[i,:]@Z[k,:].T

In [6]:
# consider confidence of social network

def convert_confidence(G):
    C=np.zeros((num_user,num_user))
    for i in range(num_user):
        for k in range(num_user):
            d_minus=np.count_nonzero(G[:,k])
            d_plus=np.count_nonzero(G[i,])
            C[i,k]=(d_minus/(d_minus+d_plus))**(0.5)*G[i,k]
    return C

In [7]:
C=convert_confidence(G)

In [8]:
def g(x):
    return 1/(1+np.exp(-x))
def g_prime(x):
    return np.exp(x)/(1+np.exp(x))**2

In [9]:
# restrict ratings to [0,1]

def restrict(R):
    for i in range(num_user):
        for j in range(num_item):
            if R[i,j]!=0:
                R[i,j]=(R[i,j]-1)/4
    return R

In [10]:
# reconstruct ratings

def f_inv(x):
    return 4*x+1

In [11]:
R_res=restrict(R)

In [12]:
def sum_vj(i):
    result=0
    for j in range(num_item):
        if R[i,j]>0:
            r=predict(i,j)
            result+=g_prime(r)*(g(r)-R_res[i,j])*V[j,:]
            
    return result

def sum_zk(i):
    result=0
    for k in range(num_user):
        if G[i,k]>0:
            r=predictc(i,k)
            result+=g_prime(r)*(g(r)-C[i,k])*Z[k,:]
    return result

def sum_ui1(j):
    result=0
    for i in range(num_user):
        if R[i,j]>0:
            r=predict(i,j)
            result+=g_prime(r)*(g(r)-R_res[i,j])*U[i,:]
            
    return result

def sum_ui2(k):
    result=0
    for i in range(num_user):
        if G[i,k]>0:
            r=predictc(i,k)
            result+=g_prime(r)*(g(r)-C[i,k])*U[i,:]
            
    return result

In [13]:
samples=[(i,j,k) for i in range(num_user) for j in range(num_item) for k in range(num_user)]

In [14]:
# initialize

U=np.random.rand(num_user,dim)/np.sqrt(dim)
V=np.random.rand(num_item,dim)/np.sqrt(dim)
Z=np.random.rand(num_user,dim)/np.sqrt(dim)

In [15]:
# train

for epoch in range(100):
    for i,j,k in samples:
        U[i,:]-=0.001*(sum_vj(i)+0.01*sum_zk(i)+0.01*U[i,:])
        V[j,:]-=0.001*(sum_ui1(j)+0.01*V[j,:])
        Z[k,:]-=0.001*(0.01*sum_ui2(k)+0.01*Z[k,:])

In [16]:
np.round(f_inv(U@V.T),2)

array([[4.7 , 1.36, 1.67, 2.15, 3.6 , 2.43, 1.88, 4.07],
       [4.65, 1.4 , 1.82, 2.03, 3.69, 2.35, 1.92, 4.01],
       [3.47, 1.23, 1.28, 1.67, 2.66, 2.24, 1.39, 3.19],
       [1.71, 1.09, 1.08, 1.36, 1.49, 1.29, 1.07, 1.63],
       [5.33, 1.36, 1.47, 2.46, 3.75, 2.72, 1.94, 4.64],
       [4.92, 1.4 , 1.82, 1.99, 3.92, 2.73, 1.86, 4.32]])